<a href="https://colab.research.google.com/github/SpyroRock/U-net-pixel-reconstruction/blob/master/U_net_pixel_reconstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/SpyroRock/U-net-pixel-reconstruction

Cloning into 'U-net-pixel-reconstruction'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 2041 (delta 3), reused 1 (delta 0), pack-reused 2033
Receiving objects: 100% (2041/2041), 250.52 MiB | 36.59 MiB/s, done.
Resolving deltas: 100% (1144/1144), done.
Checking out files: 100% (16/16), done.


In [2]:
%cd U-net-pixel-reconstruction

/content/U-net-pixel-reconstruction


In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 51.1MB 61kB/s 
     |████████████████████████████████| 307kB 52.7MB/s 
     |████████████████████████████████| 3.3MB 26.2MB/s 
     |████████████████████████████████| 573kB 57.9MB/s 
     |████████████████████████████████| 51kB 9.5MB/s 
     |████████████████████████████████| 3.3MB 54.1MB/s 
  Created wheel for keras-segmentation: filename=keras_segmentation-0.3.0-cp36-none-any.whl size=29072 sha256=e133c0f08575cd36dd798ef729d5a19688e890730f27e6b8aeef92720bb98641
  Stored in directory: /root/.cache/pip/wheels/d0/bb/c0/6aad88b38f6e46db048bed4cccb904a5897055a8ab6fbd4dfc
Successfully built keras-segmentation
ERROR: xarray 0.15.1 has requirement setuptools>=41.2, but you'll have setuptools 39.1.0 which is incompatible.
ERROR: google-auth 1.25.0 has requirement setuptools>=40.3.0, but you'll have setuptools 39.1.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: 

In [5]:
%cd U-net-pixel-reconstruction

from unet_model_construction import get_unet
import keras
from keras.models import Model
from keras.layers import Input
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import backend as K
import numpy as np
from numpy import load, save
from sklearn.model_selection import train_test_split
import pickle
import os

[Errno 2] No such file or directory: 'U-net-pixel-reconstruction'
/content/U-net-pixel-reconstruction


In [2]:
img_height = 256
img_width = 256

img_height_test = 256
img_width_test = 256

speckle_data = load('speckle_cluster_case2.npy')
print(speckle_data.shape)
speckle_labels = load('symbol_cluster_case2.npy')
print(speckle_labels.shape)

(180, 256, 256)
(180, 256, 256)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(speckle_data, speckle_labels, test_size=0.1, random_state=42)

X_train = X_train.reshape(-1, img_height, img_width, 1)
X_test = X_test.reshape(-1, img_height, img_width, 1)
input_shape = (img_height, img_width, 1)

y_train = y_train.reshape(-1, img_height_test, img_width_test, 1)
y_test = y_test.reshape(-1, img_height_test, img_width_test, 1)
input_shape_test = (img_height_test, img_width_test, 1)

print(X_train.shape)
print(y_train.shape)

(162, 256, 256, 1)
(162, 256, 256, 1)


In [4]:
# checkpoint
#checkpoint_filepath = 'weights.{epoch:02d}-{accuracy:.2f}.hdf5'
checkpoint_filepath = '/content/U-net-pixel-reconstruction/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_filepath)


model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_filepath,
                                                               save_weights_only = True,
                                                               verbose=1)

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

reconstruction = get_unet(input_shape, n_filters = 32, dropout = 0.1, batchnorm = True)

reconstruction.fit(X_train, y_train, 
                   batch_size = 10, 
                   epochs = 10500, 
                   verbose = 1, 
                   validation_data = (X_test, y_test)) # Data on which to evaluate the loss and any model metrics at the end of each epoch. 
                                                       # The model will not be trained on this data. 
                                                       # This could be a list (x_val, y_val) or a list (x_val, y_val, val_sample_weights). 
                                                       # validation_data will override validation_split.

NameError: ignored